In [1]:
import numpy as np
import pandas as pd
import seaborn as sn
import random
import copy

# Simulating amplification metrics under biased historical algorithm

Algorithmic amplification is difficult to define, partially because it is by its nature a relative metric-- we need to define amplification with respect to some baseline. Here, I look at how measures of algorithmic amplification are misleading if the baseline is the result of a biased algorithm previously. Specifically, suppose we have a population of _n_ accounts, all of which are "identical". That is, at each time point, each account draws a value from a N(0,1) and tweets it. The other accounts' preference is to see the _K_ accounts whose value is closest to theirs, with a slight preference for accounts they already follow. At each time point, they follow each accoun they have seen with probability _p_.  

I consider three types of algorthms for determing which content they see: (1) a "biased" algorithm, in which the first _num_preferred_ accounts (the "preferred users") get a boost in their ranking, (2) a "baseline" algorithm, in which every account sees content according to their known preferences as described above, and (3) reverse chron. 

Our goal is to measure how much "amplification" the biased algorithm has, which we define as the number of impressions given to the preferred users under the biased algorithm divided by the number of impressions given to the preferred users under reverse chron. 

What happens if the the "biased" algorithm is used at first? And then, we measure amplification going forward.
What happens if the "baseline" algorithm is used at first?

How does this change what we conclude about algorithmic amplification?

In [2]:
#set parameters
n = 100
p = .05
num_iter = 200
K = 10
follow_factor = 0.1
follow_prob = 0.05
num_preferred = 10

In [3]:
class simulation:
    def __init__(self, n):
        self.follows = {key : [] for key in list(range(n))}
        self.impressions = {key : [] for key in list(range(n))}
        self.n = n
        
    def get_timeline_baseline(self, values, i, K, follow_factor):
        follow_ind = np.zeros(self.n)
        follow_ind[self.follows[i]] = 1
        diff = np.abs(values[i] - values) - follow_factor*follow_ind
        top_k =  sorted(range(len(diff)), key = lambda sub: diff[sub])
        top_k.remove(i)
        return(top_k[0:K])
    
    def get_timeline_reverse_chron(self, values, i, K):
        #automatically take all followers
        top_k = self.follows[i]
        random.shuffle(top_k)
        #just switch back to an empty list in the case where top_k is empty
        if not top_k:
            top_k = []
        if (len(top_k) < K):
            diff = np.abs(values[i] - values)
            top_k = top_k + sorted(range(len(diff)), key = lambda sub: diff[sub])
            top_k.remove(i)
        top_k =  top_k[0:K] #not behaving as expected, throwing in a bunch of random shit! 
        return(top_k)
        
    def get_timeline_biased(self, values, i, K, follow_factor, num_preferred, preferred_factor):
        follow_ind = np.zeros(self.n)
        follow_ind[self.follows[i]] = 1
        diff = np.abs(values[i] - values) - follow_factor*follow_ind
        diff[0:num_preferred] -= preferred_factor
        top_k =  sorted(range(len(diff)), key = lambda sub: diff[sub])
        top_k.remove(i)
        return(top_k[:K])
    
    def run_simulation(self, num_iter, sim_type = "baseline",  K=10, follow_factor=0.1, \
                     follow_prob=0.05, num_preferred=10, preferred_factor=0.2):
        for iter in range(num_iter):
            #simulate values
            values = np.random.normal(size=self.n)
            for i in range(self.n):
                if sim_type == "baseline":
                    top_k = self.get_timeline_baseline(values, i, K, follow_factor)
                if sim_type == "biased":
                    top_k = self.get_timeline_biased(values, i, K, follow_factor, \
                                                      num_preferred, preferred_factor)
                if sim_type == "reverse_chron":
                    top_k = self.get_timeline_reverse_chron(values, i, K)
                
                #top_k.sort()
                #print(top_k)
                self.impressions[i].extend(top_k)
                follows = [j for j in top_k if np.random.uniform()<follow_prob]
                self.follows[i] = list(set(self.follows[i]).union(set(follows)))
                #this is ugly, changing from list to set back to list, maybe can make this prettier?
    
    def erase_impression_history(self):
        self.impressions = {key : [] for key in list(range(n))}
      
    def barplot_impression_counts(self, K=10, last_iters=5):
        
        df = pd.DataFrame(self.impressions).tail(K*last_iters)
        num_impressions = df.apply(lambda x: x.value_counts()).sum(axis=1)
        sn.barplot(x=num_impressions.index.values,y=num_impressions.values)

    def histogram_impression_counts(self, K=10, last_iters=5):
        print(K*last_iters)
        df = pd.DataFrame(self.impressions).tail(K*last_iters)
        num_impressions = df.apply(lambda x: x.value_counts()).sum(axis=1)
        sn.histplot(num_impressions)
    
    def get_impressions_to_preferred(self, num_preferred=10, last_iters=5):
        df = pd.DataFrame(self.impressions).tail(K*last_iters)
        num_impressions = df.apply(lambda x: x.value_counts()).sum(axis=1)
        preferred_impressions = num_impressions[num_impressions.index < num_preferred].sum()
        return(preferred_impressions)

In [4]:
#let's initialize things by running the simulation for 50 iterations under the "biased" algorithm. 
sim_pref = simulation(n)
sim_pref.run_simulation(50, "biased",K, follow_factor, follow_prob, num_preferred = 20, preferred_factor = 5)

In [5]:
#make a copy of the state of the follow graph after the initial period on which to run reverse chron going forward
sim_pref_rc = simulation(n)
sim_pref_rc.follows = copy.deepcopy(sim_pref.follows)

In [6]:
#run both the biased algorithm and reverse chron for 10 iterations going forward
sim_pref.run_simulation(10, "biased",K, follow_factor, follow_prob, num_preferred = 20, preferred_factor = 5)
sim_pref_rc.run_simulation(10, "reverse_chron")

In [7]:
#compare the number of impressions to the preferred peole under continuing with the preferred algo
print("The amplification factor under a biased previous world is : " + 
str(sim_pref.get_impressions_to_preferred(num_preferred=20, last_iters = 5)/\
sim_pref_rc.get_impressions_to_preferred(num_preferred=20, last_iters=5))
     )

#IT WORKSSSSSSS!!
#note here that each user sees 10 accounts at each iteration and 15 users are preeferred by the algorithm. 
#i've set the preferred_factor, i.e. how biased the algo is in favor of those users, to be super high
#this means in the biased algorithm users only ever see the preferred users, so their follow network only
#ever contains preferred users. Thus, we'd expect amplification to be ~ 1. I've chosen these settings for now
#so we can confirm this works, but we should fiddle with them for more compelling demos.

The amplification factor under a biased previous world is : 1.0018032458425166


In [8]:
# now let's compare to what we'd calculate amplification to be if the world had always been "fair", 
#i.e. we serve content according to the users true preferences

#initialize simulation with 50 iterations of the baseline algorithm
sim_baseline = simulation(n)
sim_baseline.run_simulation(50, "baseline",K, follow_factor, follow_prob)


In [9]:
#make a copy of the state of the follow graph after the initial period on which to run reverse chron going forward
sim_baseline_rc = simulation(n)
sim_baseline_rc.follows = copy.deepcopy(sim_baseline.follows)


In [10]:
#run the pref one for 10 more iterations with pref algo
sim_baseline.run_simulation(10, "biased",K, follow_factor, follow_prob, num_preferred = 20, preferred_factor = 5)
sim_baseline_rc.run_simulation(10, "reverse_chron")

In [11]:
#then we'd say that the biased algo is has amplification factor...
print("The amplification factor under the baseline previous world is : " + 

str(sim_baseline.get_impressions_to_preferred(10,5)/sim_baseline_rc.get_impressions_to_preferred(10,5)))


The amplification factor under the baseline previous world is : 5.04887983706721


In [ ]:
#SO IN THE CASE WHERE WE HAD THE BIASED PRIOR WORLD, WE MEASURE THE BIASED ALGORITHM AS CONTAINING NO BIAS.
#HOWEVER, IF WE COULD HAVE COMPARED IT TO A TRUE NEUTRAL STATE, WE WOULD HAVE FOUND THAT IT HAD AMPLIFICATION FACTOR OF NEARLY 5.